In [72]:
# prep
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy

nlp = spacy.load('da_core_news_sm')
vectorizer = CountVectorizer() # dan vectorizerfunktion

texts = ['Vi er utroligt beærede',
         'Vi vil gerne dele prisen med alle', 
         'Det skal vi have gjort op med.']

transformed_documents = vectorizer.fit_transform(texts) # brug vectorizer på tekster

# Konverter fittet vectorizer til array
transformed_documents_as_array = transformed_documents.toarray()

texts_processed = []

for c, text in enumerate(texts, start = 1):
    doc = nlp(text)
    
    for i, word in enumerate(doc, start = 1):
        word_info = {}
        word_info['word'] = word.text
        word_info['pos'] = word.pos_
        word_info['order'] = i
        word_info['id'] = c
        
        texts_processed.append(word_info)
        
        
tokenizer = nlp.tokenizer

df = pd.DataFrame({'text': pd.Series(texts)})
df['from'] = df['text'].apply(tokenizer)
df['to'] = df['text'].apply(tokenizer)
df = df.explode('from')
df = df.explode('to')
df['from'] = df['from'].astype('str')
df['to'] = df['to'].astype('str')

df = df.loc[df['from'] != df['to'], ['from', 'to']]

In [73]:
pd.DataFrame(transformed_documents_as_array, columns = vectorizer.get_feature_names_out())

,alle,beærede,dele,det,er,gerne,gjort,have,med,op,prisen,skal,utroligt,vi,vil
0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0
1,1,0,1,0,0,1,0,0,1,0,1,0,0,1,1
2,0,0,0,1,0,0,1,1,1,1,0,1,0,1,0


In [76]:
pd.DataFrame.from_records(texts_processed).head(10)

,word,pos,order,id
0,Vi,PRON,1,1
1,er,AUX,2,1
2,utroligt,ADV,3,1
3,beærede,VERB,4,1
4,Vi,PRON,1,2
5,vil,AUX,2,2
6,gerne,ADV,3,2
7,dele,VERB,4,2
8,prisen,NOUN,5,2
9,med,ADP,6,2


In [75]:
df.head(10)

,from,to
0,Vi,er
0,Vi,utroligt
0,Vi,beærede
0,er,Vi
0,er,utroligt
0,er,beærede
0,utroligt,Vi
0,utroligt,er
0,utroligt,beærede
0,beærede,Vi


**Disposition**
- Text clustering eksempel - betydning af pre-processing
- Intro: Tekst som data
- Fra tekst til enheder: tokenization
    - "Tegneeksempel" - hvad er vigtigt i en tekst?
    - Tokenization
    - Casing
    - Stopord
    - Stemming
    - Lemmatization
- Fra tekst til datastruktur: vectorization
    - Document-term matrix (/term-document matrix)
        - Bag-of-words
    - Tællinger
    - Binært
    - Tf-idf
    - Obs!: Sparsity
- Live coding:
    - Brug af spacy
    - Tokenizerfunktion
    - Vectorizer med egen tokenizer

# Messy data i praksis – Oprydning i tekst

# Eksempel med tekst pre-processing og klyngeanalyse (live-coding)

## Tekst som data - Udfordringer

**Det skrevne sprog er ikke entydigt**

> Jeg elsker politik

> Jeg elsker ikke politik

> Jeg elsker politik, når folk råber i munden på hinanden

> Jeg elsker politik, når folk råber i munden på hinanden. Først da kan jeg mærke, hvor passionerede politikerne er.

## Tekst som data - Udfordringer

**Det skrevne sprog er ikke entydigt**

> Tim valgte kort før Allan.

- Valgte Tim kort i et spil, før Allan valgte kort?
- Valgte Tim et eller andet lige inden, at Allan valgte noget?
- Valgte Tim kort inden han valgte Allan?

## Tekst som data - Udfordringer

**Tekstdata har altid *høj dimensionalitet* (mange variable)**

- Høj dimensionalitet gør det vanskeligere at opsummere data simpelt

- Ikke blot ordene i teksten, men også sætningskonstruktion (syntaks) og ordenes betydning (semantik) er nødvendige for at kunne opsummere teksten fyldestgørende

- Derudover kan der være faktorer, som går ud over teksten i sig selv, der kan være relevante for, hvordan den skal forstås (historisk kontekst, forfatter, medie osv.)

## Tekst som data - Udfordringer

**Tekstdata er vanskelige at standardisere**

- Gradbøjning - vælge, vælger, valgte
- Ens stavemåder - en vælger (navneord), han vælger (verbum)
- Alternative stavemåder - ressource, resurse
- Synonymer - stille, sagte
- Forskellig semantisk "vægt" - hvor relevant er ordet for teksten?

## Tekst som data - Udfordringer

**Tekstdata har ikke en given datastruktur**

- Opgjort på ord, sætninger, afsnit?
- Tællinger? Vægtning?
- Sammenlignes tekster, ord, ordforbindelser, kontekst?

## Tekster og datastrukturer

> 'Vi er utroligt beærede'

> 'Vi vil gerne dele prisen med alle'

> 'Det skal vi have gjort op med.'

## Tekster som matrix (document-term matrix)

- En række per tekst, en kolonne per ord, en celle per tælling
- (hvis kolonner som ord og rækker som tekst: term-document matrix)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>alle</th>
      <th>beærede</th>
      <th>dele</th>
      <th>det</th>
      <th>er</th>
      <th>gerne</th>
      <th>gjort</th>
      <th>have</th>
      <th>med</th>
      <th>op</th>
      <th>prisen</th>
      <th>skal</th>
      <th>utroligt</th>
      <th>vi</th>
      <th>vil</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

## Tekster som ordtabel

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>pos</th>
      <th>order</th>
      <th>id</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Vi</td>
      <td>PRON</td>
      <td>1</td>
      <td>1</td>
    </tr>
    <tr>
      <th>1</th>
      <td>er</td>
      <td>AUX</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2</th>
      <td>utroligt</td>
      <td>ADV</td>
      <td>3</td>
      <td>1</td>
    </tr>
    <tr>
      <th>3</th>
      <td>beærede</td>
      <td>VERB</td>
      <td>4</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Vi</td>
      <td>PRON</td>
      <td>1</td>
      <td>2</td>
    </tr>
    <tr>
      <th>5</th>
      <td>vil</td>
      <td>AUX</td>
      <td>2</td>
      <td>2</td>
    </tr>
    <tr>
      <th>6</th>
      <td>gerne</td>
      <td>ADV</td>
      <td>3</td>
      <td>2</td>
    </tr>
    <tr>
      <th>7</th>
      <td>dele</td>
      <td>VERB</td>
      <td>4</td>
      <td>2</td>
    </tr>
    <tr>
      <th>8</th>
      <td>prisen</td>
      <td>NOUN</td>
      <td>5</td>
      <td>2</td>
    </tr>
    <tr>
      <th>9</th>
      <td>med</td>
      <td>ADP</td>
      <td>6</td>
      <td>2</td>
    </tr>
  </tbody>
</table>

## Tekster som graf/netværk

- Hver række udgør en ordforbindelse; kolonne for hver "ende" af forbindelsen

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>from</th>
      <th>to</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Vi</td>
      <td>er</td>
    </tr>
    <tr>
      <th>0</th>
      <td>Vi</td>
      <td>utroligt</td>
    </tr>
    <tr>
      <th>0</th>
      <td>Vi</td>
      <td>beærede</td>
    </tr>
    <tr>
      <th>0</th>
      <td>er</td>
      <td>Vi</td>
    </tr>
    <tr>
      <th>0</th>
      <td>er</td>
      <td>utroligt</td>
    </tr>
    <tr>
      <th>0</th>
      <td>er</td>
      <td>beærede</td>
    </tr>
    <tr>
      <th>0</th>
      <td>utroligt</td>
      <td>Vi</td>
    </tr>
    <tr>
      <th>0</th>
      <td>utroligt</td>
      <td>er</td>
    </tr>
    <tr>
      <th>0</th>
      <td>utroligt</td>
      <td>beærede</td>
    </tr>
    <tr>
      <th>0</th>
      <td>beærede</td>
      <td>Vi</td>
    </tr>
  </tbody>
</table>

## Tekst som data - overvejelser

1. Hvordan skal tekst *bearbejdes*? (standardisering, sortering, udvælgelse)
2. Hvordan skal tekst *repræsenteres*? (datastruktur)
3. Hvordan skal tekst *analyseres*? (anvendte teknikker, metoder, modeller)

2 og 3 hænger ofte sammen (repræsentation afhænger af analysen, som skal foretages).

## NLP: Natural Language Processing

- Behandling og bearbejdning af "naturligt sprog" ved hjælp af computerteknologi
- Teknikker der involverer statistiske metoder til at forstå tekst - med eller uden lingvistiske indsigter
- Et sæt af værktøjer der tillader, at computere kan bearbejde og udlede information fra lingvistisk data (tekst eller tale)
- Krydsfelt mellem datalogi og lingvistik
- Involverer i stigende grad brug af maskinlæringsteknologi

*Eksempler fra hverdagen*: Tale-til-tekst applikationer, tekstforslag i beskeder, autokorrektur, oversættelsestjenester (Google Translate)

## Fra tekst til analyseenheder

#### Politiet har givet borgerne råd til, 
#### hvordan man kan forsøge at jage prærieulvene på flugt 
#### med larm, vandpistoler og peberspray.

## Tokenization

Opdeling af tekst til enkelte "tokens" (ordenheder)

`'Politiet har givet borgerne råd'` -> `['Politiet', 'har', 'givet', 'borgerne', 'råd']`

- Evt. frasortering af tegnsætning
- Evt. konvertering til små bogstaver (lower-casing)

**Udfordringer**:
- Egenavne i flere ord: Høje Taastrup
- Sammensatteord med bindestreger

## N-gram tokenization

Typisk referer tokenization til at opdele tekster i enkeltord.

Afhængig af formål, kan det være relevant at se på tokens som "ordpar" (bigrams) eller "ordsamlinger" (n-grams).

*Bigram eksempel:*

`'Politiet har givet borgerne råd'` -> `[('Politiet', 'har'), ('har', 'givet'), ('givet', 'borgerne'), ('borgerne', 'råd')]`

## Stemming - Ord til stammer

Typisk vil man ikke behandle ord med forskellig gradbøjning som forskellige ord

Simpleste måde at ensrette: omdanne til ordstammen

- køre  -> kør
- kører -> kør
- køren -> kør
- kørte -> kørt

Flere algoritmer til at lave stemming - *dog ikke altid enige!
- Snowball algoritme (sprogfølsom):     køren -> kør
- Porter algoritme (ikke sprogfølsom):  køren -> køren

## Lemmatization

Konverterer ordet til grammtiske stamme (alternativt til stemming)

- køre  -> køre
- kører -> køre
- køren -> køren
- kørte -> køre

Lemmatizere er af nødvendighed sprogafhængige

Findes typisk som "dictionary-modeller" (nogen har lavet opgørelser af, hvilke ord har samme grammatiske stamme)

## Stopord

Mange ord indgår i sprog uanset hvad teksten handler om (bindeord, stedord, forholdsord o.l.)

Refereres til som "stopord" - frasorteres da de typisk ikke giver indblik i, hvad tekster handler om

Stopord kan både være generelle (bindeord, stedord, forholdsord) eller kontekstspecifikke (fx 'ordfører' i referat fra Folketinget).

## Part-of-Speech tagging

Opdeler ord i ordklasser: navneord, udsagnsord, tillægsord osv.

Brugbart til at udlede meningsfulde ord i tekst (tillægsord siger typisk mere om teksten end forholdsord).

Er nødt til at være sprogafhængige *og* kontekstafhængige (fleste er baseret på maskinlæring i dag).

`'Politiet har givet borgerne råd'` -> `'Politiet_NOUN har_AUX givet_VERB borgerne_NOUN råd_NOUN'`

## Sætningskonstruktion (dependency parsing)

Udleder sætningskonstruktion: grundled, udsagnsled, genstandsled.

Hjælper til at reducere tvetydighed i tekst - "hvem gjorde hvad til hvem?"

Brugbart til at udlede meningsfulde dele af teksten (fx fokusere på visse ord, der indgår som genstandsled i sætninger).

Er nødt til at være sprogafhængige *og* kontekstafhængige (fleste er baseret på maskinlæring i dag).

`'Politiet har givet borgerne råd'` -> `'Politiet_nsubj har_aux givet_ROOT borgerne_obj råd_obj'`

## Introduktion til sprogmodeller

En sprogmodel ("language model") er kort sagt en model til at forudsige ord. Der er tale om modeller, der er trænet til at forstå forskellige sprog, og kan derfor både bruges til at producere tekst på et bestemt sprog eller analysere opbygningen af en tekst.

Lidt forsimplet er en sprogmodel en model, som er trænet til at genkende ordtyper, entiteter og sætningskonstruktion, som derved kan prædiktere disse informationer i tekst på det samme sprog.

## Introduktion til `spaCy` 

"spaCy" indeholder forskellige sprogmodeller - herunder en dansk sprogmodel.

Overordnet virker spaCy ved, at man specificerer en sprogmodel samt nogen "processors", som modellen skal indeholde. 

SpaCy's sprogmodeller indeholder blandt andet:
- Tokenizer (inddeling i enkeltord)
- Lemmatizer (konvertering til navneform)
- Part-Of-Speech tagging (POS-tagging) (identificering af ordtyper)
- Dependency parsing (sætningskonstruktion)
- Named-Entity-Recognition (NER) (udledning af "named entities", fx personer og organisationer)

## Brug af spaCy i Python

1. Indlæs sprogmodel
2. Analysér tekstykke
3. Inspicér resultater

In [117]:
import spacy

nlp = spacy.load("da_core_news_sm") # Definerer model

doc = nlp('Politiet har givet borgerne råd')

Når modellen anvendes på et stykke tekst, kan de forskellige værdier og information, som er udledt af teksten, tilgås som attributes (et attribute for token, et for lemma, et for part-of-speech-tag osv.).

In [118]:
spacy.displacy.render(doc, style='dep')

# Brug af `spaCy` i Python (live-coding)

## Fra tekst til datastruktur

Som nævnt er datastrukturer for tekster ikke givet.

Det kommer både af, at konverteringen til tokens ikke er ens, men også ud fra, hvordan ordene skal opgøres (tælles, vægtes eller andet).

Man anvender termet "text vectorization" til at referere til teknikker, der konverterer tekster til matematiske repræsentationer (datastrukturer, som man kan foretage beregninger på)

![bow](../img/bow.JPEG)

## Bag-of-words repræsentation

Mange modeller bygger på "bag-of-words" repræsentationer af tekst (bag-of-words modeller).

Optales bag-of-words, da denne struktur ikke tager højde for sætningskonstruktion (syntaks) eller semantik.

Eksempel på bag-of-words struktur: Document-term matrix:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>alle</th>
      <th>beærede</th>
      <th>dele</th>
      <th>det</th>
      <th>er</th>
      <th>gerne</th>
      <th>gjort</th>
      <th>have</th>
      <th>med</th>
      <th>op</th>
      <th>prisen</th>
      <th>skal</th>
      <th>utroligt</th>
      <th>vi</th>
      <th>vil</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
  </tbody>
</table>


**Udfordringer med bag-of-words repræsentation**

- Tager ikke højde for sætningskonstruktion og semantik
- Giver ofte datastrukturer med høj "sparsity" (mange tomme celler).

## Ordtællinger

## Binær tælling

## Tf-idf vægtning af ord